In [6]:
import os
import requests
import pandas as pd
import numpy as np
from timeit import default_timer as timer
import datetime
from tika import parser
from unidecode import unidecode
import pickle
import pdfkit
from utils import get_chromedriver, download_from_drive_dropbox

In [2]:
df = pd.read_csv('.\\Data and papers\\tord_v3.csv', sep = ";")
df

,id,name,token,country,is_ico,is_ieo,is_sto,ico_start,ico_end,price_usd,...,platform,accepting,link_white_paper,linkedin_link,github_link,website,rating,teamsize,Coinmarketcap_identifier,ERC20
0,1,WINSSHI,WNS,India,1,0,0,2020-08-10,2020-12-31,0.01,...,0,"BTC, ETH, and Fiat currency",NaN,https://www.linkedin.com/company/bitfinex-limi...,NaN,NaN,4.0,NaN,NaN,1.0
1,2,Tycoon,Tycoon Token / TYC,Cyprus,1,0,0,2020-08-01,2020-12-31,0.1,...,1,"ETH, BTC",https://www.tycoon.io/whitepaper.pdf,https://www.linkedin.com/company/tycoonio/about/,https://www.github.com/tycoon?tab=repositories,https://www.tycoon.io?utm_source=icobench,4.8,14.0,NaN,1.0
2,3,Mindsync,MAI,UK,1,0,0,2019-03-01,2020-12-31,0.14000000000000001,...,0,"ETH, BTC, LTC",https://mindsync.ai/docs/whitepaper.pdf,https://www.linkedin.com/company/mindsync/,NaN,NaN,4.5,14.0,NaN,1.0
3,4,PointPay,PXP,UK,1,0,0,2020-06-25,2021-01-31,0.1,...,0,"BTC, ETH, XRP, BNB, LTC, XLM, USDT, BCH, TRX, EOS",https://cdn.pointpay.io/WhitePaper__en.pdf?cac...,https://www.linkedin.com/company/pointpay/about/,NaN,NaN,4.7,43.0,5640.0,1.0
4,5,LOHN,LOHN,Seychelles,1,0,0,NaN,NaN,0.06,...,1,ETH,https://lohncontrol.com/down/LOHN-white-paper-...,https://www.linkedin.com/showcase/lohncontrol,https://www.github.com/lohn7?tab=repositories,https://lohncontrol.com?utm_source=icobench,4.4,16.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410,6423,Thousand Token,THOUSAND,Colombia,1,0,0,NaN,NaN,0.02,...,0,"Waves, BTC, ETH, USDN",https://drive.google.com/file/d/1j3J4JXBF4xsps...,TBD,NaN,NaN,NaN,1.0,NaN,NaN
6411,6424,Cryptocurrency Market,CCM,Unknown,1,0,0,NaN,NaN,0.25,...,0,ETH,https://cryptocurrency-market.co/whitepaper.pdf,TBD,NaN,NaN,NaN,3.0,NaN,NaN
6412,6425,IQ Token,IQ,Unknown,1,0,0,NaN,NaN,NaN,...,0,ETH,NaN,https://www.linkedin.com/in/e3-t-61654018b/,NaN,NaN,NaN,NaN,NaN,NaN
6413,6426,Seseral,SSL,Unknown,1,0,0,NaN,NaN,12,...,0,"TRX, BTC, ETH, USDT",https://seseral.finance/whitepaper,TBD,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
download_df = df[['id', 'link_white_paper']].dropna().reset_index(drop = True)
download_df['Status'] = ""
download_df['Error'] = ""
download_df['Path_Original'] = ""

## Download pdf

In [105]:
HEADERS = {"User-Agent": "Chrome/51.0.2704.103"}
PDF_FOLDER = 'Whitepaper\\Original'

start = timer()
OK_count = 0
for index, row in download_df.iterrows():
    
    url = row['link_white_paper']
    file_name = os.path.join(PDF_FOLDER, 'ID_' + str(row['id']) + '.pdf')

    print('Downloading ' + str(index + 1) + ' / ' + str(len(download_df)) + '  - Total OK: ' + str(OK_count), end = '\r')
    
    try:
        # connect
        response = requests.get(url, headers = HEADERS)

        # check response and save pdf
        if response.status_code == 200:
            with open(file_name, "wb") as f:
                f.write(response.content)
            download_df.loc[index, 'Status'] = 'OK'
            download_df.loc[index, 'Path_Original'] = os.path.join(os.getcwd(), file_name)
            OK_count += 1
        else:
            download_df.loc[index, 'Status'] = response.status_code
            
    except Exception as e:
        download_df.loc[index, 'Status'] = 'ERROR'
        download_df.loc[index, 'Error'] = e
    
    # save checkpoint
    download_df.to_csv('./Checkpoints/whitepaper_download.csv', index=False, sep=';')
            
print('\nTotal elapsed time:', str(datetime.timedelta(seconds=round(timer()-start))))

# save results
download_df.to_csv('./Results/00a_whitepaper_download_original.csv', index=False, sep=';')

KeyboardInterrupt: 

## Check downloaded files

In [85]:
download_df = pd.read_csv('./Results/00a_whitepaper_download_original.csv', sep = ";")
print(download_df['Status'].value_counts())

final_df = download_df.copy()
final_df = final_df[final_df['Status'] == 'OK'].reset_index(drop = True)
final_df['Path_Recovered'] = ""
final_df['Path_txt'] = ""
final_df['Status_txt'] = ""
final_df['Length_txt'] = 0
final_df['Length_txt_clean'] = 0
final_df['Content_txt'] = ""
final_df['Metadata'] = ""

ERROR    2222
OK       2058
404      1180
403       124
410        73
522        73
500        20
503        18
521        12
530        11
502         7
526         5
301         5
523         4
525         3
400         3
520         2
429         2
504         2
401         1
204         1
423         1
408         1
Name: Status, dtype: int64


In [86]:
# Parse pdf and convert to txt
TXT_FOLDER = "Whitepaper\\Converted_to_txt"

start = timer()
for index, row in final_df.iterrows():
    
    file_path = row['Path_Original']
    file_path_txt = os.path.join(os.getcwd(), TXT_FOLDER, 'ID_' + str(row['id']) + '.txt')
    
    print('Parsing ' + str(index + 1) + ' / ' + str(len(final_df)), end = '\r')
    
    # open pdf
    parsed_pdf = parser.from_file(file_path)
    
    # extract text and metadata
    txt = parsed_pdf['content']
    meta = parsed_pdf['metadata']
    
    final_df.loc[index, 'Status_txt'] = parsed_pdf['status']
    final_df.loc[index, 'Length_txt'] = len(txt) if txt is not None else 0    # case of pdf saved as image
    final_df.loc[index, 'Length_txt_clean'] = len(txt.replace('\n','').replace(' ', '')) if txt is not None else 0    # clear whitespace to filter empty files
    final_df.at[index, 'Content_txt'] = meta['Content-Type']
    final_df.loc[index, 'Metadata'] = [meta]
    
    # save txt
    if txt is not None:
        final_df.loc[index, 'Path_txt'] = file_path_txt
        with open(file_path_txt, 'w') as f:
            f.write(unidecode(txt))
    
    # save checkpoint
    if index % 300 == 0 or index == (len(final_df) - 1):
        final_df.drop(columns='Metadata').to_csv('./Checkpoints/whitepaper_parsing.csv', index=False, sep=';')
        with open('./Checkpoints/whitepaper_parsing.pickle', 'wb') as handle:
                    pickle.dump(final_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
print('\nTotal elapsed time:', str(datetime.timedelta(seconds=round(timer()-start))))

# save results
final_df.drop(columns='Metadata').to_csv('./Results/00b_whitepaper_parsing.csv', index=False, sep=';')

Parsing 2058 / 2058
Total elapsed time: 0:05:51


In [94]:
# stats
with open('./Checkpoints/whitepaper_parsing.pickle', 'rb') as handle:
    final_df = pickle.load(handle)

# status
print('-- Status --\n')
print(final_df['Status'].value_counts())
# text length
print('\n\n-- Text length --\n')
print(final_df[['Length_txt', 'Length_txt_clean']].describe(percentiles = [.5, .10, .15, .2, .25, .3, .4, .75, .95]))
# pdf content
cnt = final_df['Content_txt'].values
cnt_single = [x for x in cnt if type(x) == str]
cnt_multiple = [x for x in cnt if type(x) != str]
print('\n\n-- Pdf content --')
print('\n- Single values (' + str(len(cnt_single)) + '):')
print(pd.DataFrame({'Content_txt': cnt_single})['Content_txt'].value_counts())
print('\n- Multiple values (' + str(len(cnt_multiple)) +'):')
print(pd.DataFrame({'aa': [len(x) for x in cnt_multiple]})['aa'].value_counts())

-- Status --

OK    2058
Name: Status, dtype: int64


-- Text length --

          Length_txt  Length_txt_clean
count    2058.000000       2058.000000
mean    24565.056851      18976.979592
std     36113.994331      28853.123588
min         0.000000          0.000000
10%        70.000000         30.000000
15%        79.000000         42.000000
20%       109.400000         58.000000
25%       281.000000         93.250000
30%       475.200000        252.000000
40%      2765.600000       1403.200000
50%      7179.000000       4258.000000
75%     40048.750000      31840.000000
95%     93873.350000      75264.150000
max    485969.000000     411334.000000


-- Pdf content --

- Single values (1969):
application/pdf                                778
text/html; charset=UTF-8                       761
text/html; charset=ISO-8859-1                  281
application/xhtml+xml; charset=UTF-8            89
application/xhtml+xml; charset=ISO-8859-1       33
text/plain; charset=ISO-8859-1            

## Try to recover empty pdf

In [2]:
Length_txt_clean_thrsh = 4000     # threshold for maximum non-empty characters in parsed txt file


with open('./Checkpoints/whitepaper_parsing.pickle', 'rb') as handle:
    final_df_recover = pickle.load(handle)
final_df_recover['Recover_action'] = "SKIP"
final_df_recover['Recover_Length_txt'] = -1
final_df_recover['Recover_Length_txt_clean'] = -1
final_df_recover['Recover_Path_txt'] = ""

In [3]:
RECOVER_FOLDER = "Whitepaper\\Recovered"
PATH_TO_WHHTMLTOPDF = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe' # define path to wkhtmltopdf.exe, see https://python-bloggers.com/2022/06/convert-html-to-pdf-using-python/
CHROMEDRIVER_PATH = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\ICOs\WebDriver\chromedriver"
TEMP_DOWNLOAD_FOLDER = "C:\\Users\\Alessandro Bitetto\\Downloads\\UniPV\\ICOs\\temp_download"
ID_TO_SKIP = ['494', '1369', '2527', '2548', '4100', '6138']


# Point pdfkit configuration to wkhtmltopdf.exe
config = pdfkit.configuration(wkhtmltopdf=PATH_TO_WHHTMLTOPDF)
start = timer()
for index, row in final_df_recover.iterrows():

    print('Recovering ' + str(index + 1) + ' / ' + str(len(final_df_recover)) + ' (' + 'ID_' + str(row['id']) + ')', end = '\r')

    file_path_pdf = os.path.join(os.getcwd(), RECOVER_FOLDER, 'ID_' + str(row['id']) + '.pdf')
    file_path_txt = os.path.join(os.getcwd(), RECOVER_FOLDER, 'ID_' + str(row['id']) + '.txt')
    original_len_txt = row['Length_txt']
    original_len_txt_clean = row['Length_txt_clean']
    url = row['link_white_paper']
    
    # check if txt length is below threshold
    if row['Length_txt_clean'] <= Length_txt_clean_thrsh:
        
        if str(row['id']) in ID_TO_SKIP:
            pass
        
        ##### download pdf from google.drive or dropbox

        elif any(x in url for x in ['google', 'goo.gl', 'dropbox']):

            source = 'drive' if any(x in url for x in ['google', 'goo.gl']) else 'dropbox'

            out = download_from_drive_dropbox(chromedriver_path=CHROMEDRIVER_PATH, download_url=url,
                                              download_folder=TEMP_DOWNLOAD_FOLDER, temp_folder=TEMP_DOWNLOAD_FOLDER,
                                              pdf_name='ID_' + str(row['id']) + '.pdf',
                                              move_folder=os.path.join(os.getcwd(), RECOVER_FOLDER), source=source)

            if out == "ok":

                # open pdf
                parsed_pdf = parser.from_file(file_path_pdf)

                # extract text and metadata
                txt = parsed_pdf['content']
                meta = parsed_pdf['metadata']

                len_txt_clean = len(txt.replace('\n','').replace(' ', '')) if txt is not None else 0 

                # update files
                if len_txt_clean > original_len_txt_clean:

                    final_df_recover.loc[index, 'Recover_action'] = 'DOWNLOAD ' + source.upper() + ' - OK'
                    final_df_recover.loc[index, 'Path_Recovered'] = file_path_pdf
                    final_df_recover.loc[index, 'Recover_Path_txt'] = file_path_txt
                    final_df_recover.loc[index, 'Recover_Length_txt'] = len(txt)
                    final_df_recover.loc[index, 'Recover_Length_txt_clean'] = len_txt_clean
                    final_df_recover.at[index, 'Content_txt'] = meta['Content-Type']
                    final_df_recover.loc[index, 'Metadata'] = [meta]

                    # save txt
                    with open(file_path_txt, 'w') as f:
                        f.write(unidecode(txt))

            else:
                final_df_recover.loc[index, 'Recover_action'] = 'DOWNLOAD ' + source.upper() + ' - ' + out
        
        
        ##### try to download pdf from html page   e.g. https://www.quasa.io/white-paper
        
        else:

            try:
                # Convert Webpage to PDF
                pdfkit.from_url(url, output_path=file_path_pdf, configuration=config)

                # open pdf
                parsed_pdf = parser.from_file(file_path_pdf)

                # extract text and metadata
                txt = parsed_pdf['content']
                meta = parsed_pdf['metadata']

                len_txt_clean = len(txt.replace('\n','').replace(' ', '')) if txt is not None else 0 

                # update files
                if len_txt_clean > original_len_txt_clean:

                    final_df_recover.loc[index, 'Recover_action'] = "CONVERT FROM HTML"
                    final_df_recover.loc[index, 'Path_Recovered'] = file_path_pdf
                    final_df_recover.loc[index, 'Recover_Path_txt'] = file_path_txt
                    final_df_recover.loc[index, 'Recover_Length_txt'] = len(txt)
                    final_df_recover.loc[index, 'Recover_Length_txt_clean'] = len_txt_clean
                    final_df_recover.at[index, 'Content_txt'] = meta['Content-Type']
                    final_df_recover.loc[index, 'Metadata'] = [meta]

                    # save txt
                    with open(file_path_txt, 'w') as f:
                        f.write(unidecode(txt))
            except:
                final_df_recover.loc[index, 'Recover_action'] = "CONVERT FROM HTML - FAILED"

    else:
        final_df_recover.loc[index, 'Recover_action'] = "KEEP ORIGINAL"
        
    # save checkpoint
    if index % 300 == 0 or index == (len(final_df_recover) - 1):
        final_df_recover.drop(columns='Metadata').to_csv('./Checkpoints/whitepaper_recover.csv', index=False, sep=';')
        with open('./Checkpoints/whitepaper_recover.pickle', 'wb') as handle:
                    pickle.dump(final_df_recover, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
print('\nTotal elapsed time:', str(datetime.timedelta(seconds=round(timer()-start))))

# save results
final_df_recover.drop(columns='Metadata').to_csv('./Results/00c_whitepaper_recover.csv', index=False, sep=';')

2022-10-23 04:34:20,583 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Recovering 2058 / 2058 (ID_6423)
Total elapsed time: 6:56:45


In [19]:
# recover actions stats
with open('./Checkpoints/whitepaper_recover.pickle', 'rb') as handle:
    final_df_recover = pickle.load(handle)

print('-- Recover actions --\n')
print(final_df_recover['Recover_action'].value_counts())

-- Recover actions --

KEEP ORIGINAL                            1049
SKIP                                      345
CONVERT FROM HTML                         249
DOWNLOAD DRIVE - OK                       149
CONVERT FROM HTML - FAILED                124
DOWNLOAD DRIVE - page not available       114
DOWNLOAD DROPBOX - page not available      17
DOWNLOAD DROPBOX - OK                       9
DOWNLOAD DRIVE - out of time                2
Name: Recover_action, dtype: int64


In [20]:
# set final path for txt files
recover_to_keep = ['CONVERT FROM HTML', 'DOWNLOAD DRIVE - OK', 'DOWNLOAD DROPBOX - OK']

final_df_recover['Final_Path_txt'] = np.where(final_df_recover['Recover_action'] == 'KEEP ORIGINAL', final_df_recover['Path_txt'], '')
final_df_recover['Final_Path_txt'] = np.where(final_df_recover['Recover_action'].isin(recover_to_keep), final_df_recover['Recover_Path_txt'], final_df_recover['Final_Path_txt'])
final_df_recover['Final_Length_txt'] = np.where(final_df_recover['Recover_action'] == 'KEEP ORIGINAL', final_df_recover['Length_txt'], '')
final_df_recover['Final_Length_txt'] = np.where(final_df_recover['Recover_action'].isin(recover_to_keep), final_df_recover['Recover_Length_txt'], final_df_recover['Final_Length_txt'])
final_df_recover['Final_Length_txt_clean'] = np.where(final_df_recover['Recover_action'] == 'KEEP ORIGINAL', final_df_recover['Length_txt_clean'], '')
final_df_recover['Final_Length_txt_clean'] = np.where(final_df_recover['Recover_action'].isin(recover_to_keep), final_df_recover['Recover_Length_txt_clean'], final_df_recover['Final_Length_txt_clean'])


final_df_recover.drop(columns='Metadata').to_csv('./Results/00d_whitepaper_final.csv', index=False, sep=';')

print('Total available whitepapers:', sum(final_df_recover['Final_Path_txt'] != ''))
with open('./Checkpoints/whitepaper_final.pickle', 'wb') as handle:
    pickle.dump(final_df_recover, handle, protocol=pickle.HIGHEST_PROTOCOL)

Total available whitepapers: 1456


In [21]:
final_df_recover.head(5)

,id,link_white_paper,Status,Error,Path_Original,Path_Recovered,Path_txt,Status_txt,Length_txt,Length_txt_clean,Content_txt,Metadata,Recover_action,Recover_Length_txt,Recover_Length_txt_clean,Recover_Path_txt,Final_Path_txt,Final_Length_txt,Final_Length_txt_clean
0,2,https://www.tycoon.io/whitepaper.pdf,OK,NaN,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,200,58501,48450,application/pdf,"{'Content-Type': 'application/pdf', 'Last-Modi...",KEEP ORIGINAL,-1,-1,,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,58501,48450
1,3,https://mindsync.ai/docs/whitepaper.pdf,OK,NaN,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,200,73801,61180,application/pdf,"{'Content-Type': 'application/pdf', 'Creation-...",KEEP ORIGINAL,-1,-1,,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,73801,61180
2,5,https://lohncontrol.com/down/LOHN-white-paper-...,OK,NaN,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,200,74525,59500,application/pdf,"{'Author': 'Vali', 'Content-Type': 'applicatio...",KEEP ORIGINAL,-1,-1,,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,74525,59500
3,6,https://emanate.live/pdf/mn8-whitepaper-v12.pdf,OK,NaN,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,200,245,166,application/pdf,"{'Content-Type': 'application/pdf', 'Creation-...",CONVERT FROM HTML,365,292,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,365,292
4,7,https://hashbon.com/info/whitepaper_eng.pdf,OK,NaN,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,200,11197,9114,application/pdf,"{'Content-Type': 'application/pdf', 'Creation-...",KEEP ORIGINAL,-1,-1,,C:\Users\Alessandro Bitetto\Downloads\UniPV\IC...,11197,9114
